In [1]:
# -------------------------------
# 🧠 Smart Tech Assistant
# A dual-model chatbot using OpenAI & Ollama with Gradio UI
# -------------------------------

import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

# Load environment variables (API keys from .env file)
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# Model identifiers
MODEL_OPENAI = "gpt-4o-mini"
MODEL_OLLAMA = "llama3.2"

# Create OpenAI client (official OpenAI)
openai_official = OpenAI(api_key=api_key)

# Create OpenAI-compatible client for local Ollama instance
openai_ollama = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

# Default assistant instruction
DEFAULT_SYSTEM_PROMPT = (
    "You are a helpful technical teaching assistant. Analyze the user's question carefully, "
    "respond in detail using markdown formatting, and include references where relevant."
)

# Build the system message for LLM
def build_initial_prompt(system_prompt):
    return {"role": "system", "content": system_prompt}

# Generate the assistant response using chosen model
def generate_response(model_choice, chat_history):
    if not chat_history:
        return "Please enter a question."

    # Select client and model
    client = openai_official if model_choice == "OpenAI GPT-4o-mini 🔵" else openai_ollama
    model = MODEL_OPENAI if model_choice == "OpenAI GPT-4o-mini 🔵" else MODEL_OLLAMA

    try:
        response = client.chat.completions.create(
            model=model,
            messages=chat_history,
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"❌ Error: {str(e)}"

# Core chat interaction function
def chat_interface(user_input, model_choice, system_prompt, history):
    if history is None:
        history = []

    messages = [build_initial_prompt(system_prompt)]
    for item in history:
        messages.append(item)
    messages.append({"role": "user", "content": user_input})

    # Show thinking placeholder
    history.append({"role": "user", "content": user_input})
    history.append({"role": "assistant", "content": "⏳ Thinking..."})
    yield history, history

    answer = generate_response(model_choice, messages)
    history[-1]["content"] = answer
    yield history, history


# -------------------------------
# 🎨 Gradio UI
# -------------------------------

with gr.Blocks(title="🧠 Smart Tech Assistant", theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    <div style="text-align: center; font-size: 1.5em; font-weight: bold;">
    🤖 Ask Me Anything – Powered by OpenAI & Ollama
    </div>
    """)
    
    with gr.Row():
        model_choice = gr.Dropdown(
            ["OpenAI GPT-4o-mini 🔵", "Ollama LLaMA 3.2 🟢"],
            label="Select your preferred model",
            value="OpenAI GPT-4o-mini 🔵"
        )
        clear_btn = gr.Button("🧹 Clear Chat", variant="secondary")

    system_prompt = gr.Textbox(
        label="System Prompt (Optional)",
        value=DEFAULT_SYSTEM_PROMPT,
        lines=2
    )

    chatbot = gr.Chatbot(label="💬 Assistant Chat", show_copy_button=True, type="messages")
    user_input = gr.Textbox(placeholder="Type your technical question here...", show_label=False)
    state = gr.State([])

    with gr.Row():
        submit_btn = gr.Button("🚀 Ask", variant="primary")

    # Event handlers
    submit_btn.click(chat_interface,
                     inputs=[user_input, model_choice, system_prompt, state],
                     outputs=[chatbot, state])
    user_input.submit(chat_interface,
                      inputs=[user_input, model_choice, system_prompt, state],
                      outputs=[chatbot, state])
    clear_btn.click(lambda: ([], []), outputs=[chatbot, state])

# Run app in notebook
demo.launch(inline=True, share=True)



* Running on local URL:  http://127.0.0.1:7863

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
